In [1]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')  # Download the tokenizer data

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()  # Initialize the PorterStemmer correctly

import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gkk46\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [7]:
nltk.download('punkt')
words=[]
classes=[]
documents=[]
ignore=['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # Add to documents
        documents.append((w, intent['tag']))
        
        # Add to classes if the tag is not already present
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gkk46\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes",classes)
print (len(words), "unique stemmed words", words)


27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [9]:
training = []
output = []
output_empty = [0] * len(classes)

# Create the training data
for doc in documents:
    bag = []
    pattern_words = doc[0]
    
    # Stem and lowercase each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # Create the bag of words
    for w in words:
        bag.append(1 if w in pattern_words else 0)
        
    # Create the output row, setting the index of the current class to 1
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    # Add the bag of words and the output row to the training data
    training.append([bag, output_row])

# Shuffle the training data to ensure randomness
random.shuffle(training)

# Convert training data into two separate lists: train_x and train_y
train_x = [row[0] for row in training]  # input features (bag of words)
train_y = [row[1] for row in training]  # output labels (one-hot encoded)

# Optionally, you can convert them into NumPy arrays if you need that for TensorFlow
train_x = np.array(train_x)
train_y = np.array(train_y)

# Now your training data is ready to use in a neural network model


In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape = (len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss ='categorical_crossentropy', metrics=['accuracy'])

c:\Users\gkk46\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0231 - loss: 2.1126  11
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0231 - loss: 2.0692   
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1819 - loss: 1.9996
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1444 - loss: 2.0275  
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1301 - loss: 2.0112    
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2389 - loss: 1.9917  
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2014 - loss: 2.0040  
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2394 - loss: 1.9813  
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3227 - loss: 1.9186  
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2731 - loss: 1.9217  
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3102 - loss: 1.8698  
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

In [15]:
import os

In [20]:
model_dir = "models"  # Specify your directory
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model in the SavedModel format
model.save(os.path.join(model_dir, "my_model.h5"))  # Save as a single file

In [18]:
import pickle
pickle.dump({'words': words, 'classes':classes}, open("training_data","wb"))

In [26]:
from keras.models import load_model
model = load_model(r"\github\python projects\ML projects\ML-1\Simple-AI-Chatbot\models\my_model.h5")

In [27]:
data = pickle.load (open("training_data","rb"))
words = data['words']
classes = data['classes']

In [30]:
with open ('intents.json') as json_data:
    intents = json.load(json_data)

In [31]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [32]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [33]:
response('Where are you located?')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
You can visit India to meet us


In [34]:
response('That is helpful')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
My pleasure


In [35]:
response('Bye')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Have a nice day


In [36]:
response("Who are you?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Thanks for asking. I am Mayank Bajaj, coder by profession but ML enthusiast by passion.
